In [ ]:
#pip install numpy pandas scikit-learn tensorflow keras rasterio matplotlib

In [58]:
# Importar bibliotecas necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt

In [62]:
# Función para cargar DEM
def load_dem(file_path):
    with rasterio.open(file_path) as src:
        dem = src.read(1)
        transform = src.transform
        crs = src.crs
    return dem, transform, crs

# Función para alinear DEMs
def align_dem(dem, src_transform, src_crs, template_transform, template_crs, template_shape):
    aligned_dem = np.empty(shape=template_shape, dtype=np.float32)
    
    reproject(
        source=dem,
        destination=aligned_dem,
        src_transform=src_transform,
        src_crs=src_crs,
        dst_transform=template_transform,
        dst_crs=template_crs,
        resampling=Resampling.nearest
    )
    
    return aligned_dem

In [63]:
# Cargar los DEMs
dem_drone, drone_transform, drone_crs = load_dem('RASTER/DEM_DRONE.tif')
dem_landviewer, landviewer_transform, landviewer_crs = load_dem('RASTER/DEM_LANDVIEWER.tif')

# Alinear el DEM del drone con el de Landviewer
aligned_drone_dem = align_dem(
    dem_drone, 
    drone_transform, 
    drone_crs, 
    landviewer_transform, 
    landviewer_crs, 
    dem_landviewer.shape
)

# Reemplazar valores nulos por -9999
dem_landviewer = np.nan_to_num(dem_landviewer, nan=-9999)
aligned_drone_dem = np.nan_to_num(aligned_drone_dem, nan=-9999)

In [77]:
dem_landviewer.shape

(2324, 2425)

In [64]:
# Crear el DataFrame para el modelo
data = {
    'drone': aligned_drone_dem.flatten(),
    'landviewer': dem_landviewer.flatten()
}

df = pd.DataFrame(data)
df = df[df['drone'] != -9999]
df = df[df['landviewer'] != -9999]

X = df[['landviewer']]
y = df['drone']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [65]:
# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'MSE Random Forest: {mse_rf}')

MSE Random Forest: 182894357.01376447


In [ ]:
# Deep Learning Model
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=3, batch_size=10, verbose=1)

y_pred_dl = model.predict(X_test)
mse_dl = mean_squared_error(y_test, y_pred_dl)
print(f'MSE Deep Learning: {mse_dl}')

In [ ]:
# Visualization
plt.figure(figsize=(10, 5))
plt.scatter(X_test, y_test, color='blue', label='True values')
plt.scatter(X_test, y_pred_rf, color='green', label='RF predictions')
plt.scatter(X_test, y_pred_dl, color='red', label='DL predictions')
plt.legend()
plt.show()